<a href="https://colab.research.google.com/github/rskrisel/factiva_dataframe/blob/main/Create_spreadsheet_Factiva_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collecting news data from Factiva and saving it in a Dataframe

**For detailed instructions on how to collect Factiva data, review this [step by step guide with images](https://docs.google.com/document/d/1C9crxk6kdYJZqU0bEx1UZMxFZpowQe3ESQO_hA1wUK0/edit?usp=sharing)**

* Step 1: Locate the `Factiva database` on the `Columbia University website`

* Step 2: Enter your search term(s) and click `Search`

* Step 3: Click `Display Options` and select `Full Article/Report plus Indexing`

* Step 4: Select `Duplicates: Identical`

* Step 5: Click the ✅ to select the articles you want to save

* Step 6: Click the `print` icon, then select `Article Format`

* Step 7: If your `print dialogue opens`, click `cancel`

* Step 8: From the `print preview page`, type `command` + `u` from your keyboard. An HTML view should open

* Step 9: From the HTML view, `select all` the text and then `copy` it

* Step 10: `Paste` the HTML code into the `html_code` variable in the `Google Colab worksheet`













In [ ]:
html_code = """ Paste html here """

In [ ]:
import glob # used to find all the file paths that match a specified pattern
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

create new factiva folder

In [ ]:
import os

# ✅ Give your folder a clear name — e.g., use your initials or project topic
FOLDER_NAME = "factiva_ner_project"

# Build the full path
DRIVE_ROOT = "/content/drive/MyDrive"
path = os.path.join(DRIVE_ROOT, FOLDER_NAME)

# Create the folder if it doesn’t exist
os.makedirs(path, exist_ok=True)

print(f"✅ Folder ready at: {path}")


Next, you want to save the contents of the `html_code` variable into a `.htm` file:

In [ ]:

# Write the HTML code to the file
with open(f"{path}/factiva.htm", 'w') as file: #replace with your path
    file.write(html_code)


# # For a list of variables with HTML content
# html_list = [html_code1, html_code2, html_code3]  # Replace with your actual variables

# # Iterate over the list and write each HTML content to a separate file
# for i, html_code in enumerate(html_list):
#     file_path = f"/content/drive/MyDrive/Factiva/factiva_{i}.htm"  # Replace with your path
#     with open(file_path, 'w') as file:
#         file.write(html_code)



In the following line, you will look for all `.htm` files in your Factiva folder (and its subfolders, if any) located at the specified path on your Google Drive and return them as a list.

- glob.glob is a function that finds all the files that match a specific pattern. In this case, it looks for files inside a folder called Factiva that have the .htm extension.
- The part `/content/drive/MyDrive/Factiva/*.htm` is the path where it will look for the files. You would replace this with the path where your own files are located. The *.htm means it will find all files ending with .htm (which are likely HTML files).
- recursive = True allows the function to search within subdirectories inside the Factiva folder as well.




In [ ]:
files = glob.glob(f"{path}/*.htm", recursive = True) #replace with your path

In [ ]:
files

The following `for loop` starts with an empty list. It then goes through each HTML file in the files list, reads any tables found in those files, and adds them to the empty_list.

In this case, the goal is to have a list where each element is a dataframe, not a list of lists. Since `pd.read_html()` returns a list of dataframes for each file, `extend` is used to merge those dataframes directly into `empty_list` so that it contains all the dataframes in one flat structure.

If you used `append`, you would end up with a nested structure where each element is a list of dataframes, which is likely not what you want.

In [ ]:
empty_list = []
for file in files:
    data = pd.read_html(file, index_col = 0) #reads the HTML content of the file and tries to find any tables inside it.
    empty_list.extend(data) # The extend() method is used to add the data (which is a list of dataframes) from the current file to empty_list.

In [ ]:
empty_list

Let's create a variable, `frames`, which contains a dataframe where all relevant dataframes (those containing 'HD' in their index) are combined and flipped.

In the next line of code, we will accomplish the following:
- Look through all the dataframes in `empty_list` and selects only the ones where `HD` is found in the index.
- Concatenate those dataframes side by side (combining their columns).
- Finally, transpose the resulting dataframe, flipping the rows and columns, and assigning it to the variable `frames`.


Breaking it down:
- `[l for l in empty_list if 'HD' in l.index.values]:`
1. This is a list comprehension. It goes through each item `l` in `empty_list` (which contains dataframes).
2. For each dataframe `l`, it checks if `'HD'` is present in the index values of that dataframe `(l.index.values)`.
3. If `'HD'` is found in the index of a dataframe, that dataframe is included in the resulting list. If `'HD'` is not found, that dataframe is ignored.

- `pd.concat([...], axis=1)`:

1. this concatenates (joins) all the dataframes that contain `'HD'` in their index. The `axis=1` argument means the dataframes will be concatenated side by side, meaning their columns will be combined.The result is a new dataframe where the data from each matching dataframe is merged by columns.

- `.T`:
1. This is a shorthand for "transpose," which flips the rows and columns of the resulting dataframe.
2. After concatenating the dataframes side by side, `.T` switches the rows and columns, so what were previously columns are now rows, and vice versa.

In [ ]:
frames = pd.concat([l for l in empty_list if 'HD' in l.index.values], axis=1).T

In [ ]:
frames

Take a look at the columns in our `frames` dataframe. The columns correspond to the index on the left column of the print view of articles in Factiva.

In [ ]:
frames.columns

Next, let's rename certain columns to more meaningful or readable names (e.g., 'HD' becomes 'Headline', 'PD' becomes 'Publication_Date', etc.) and select only the relevant columns we want to keep.

In [ ]:
frames.rename(columns = {'HD': 'Headline',
                         'PD': 'Publication_Date','SN': 'Source_Name', 'LP': 'Lead Paragraph',
                          'TD': 'Body',
                         'BY':'Author_Name'}, inplace=True)

frames = frames[['Headline', 'Publication_Date', 'Source_Name', 'Lead Paragraph', 'Body', 'Author_Name']]


Let's make sure our `Publication_Date` column is in datetime format.

In [ ]:
frames['Publication_Date'] = pd.to_datetime(frames['Publication_Date'])
frames.sort_values(by='Publication_Date', inplace=True)

Next, we want to combine the text from the `Lead Paragraph` and the `Body` columns so we have the full article in a single cell.

In [ ]:
frames['CombinedText'] = frames['Lead Paragraph'] + " " + frames['Body']

In [ ]:
frames

Let's reset the index so it's the standard [0:] index, in ascending order.

In [ ]:
df = frames.reset_index()

In [ ]:
save_path = os.path.join(path, "factiva.csv")
df.to_csv(save_path, index=False)

Next, we will us a code that loops through each row in the DataFrame `df`, creates a unique text file for each row, and writes the content from the `CombinedText` column into that file. If the `CombinedText` value is missing (i.e., `NaN`), it writes an empty string instead. The filenames are generated dynamically based on the row index.

In essence, it saves the text content from each row in the DataFrame as individual text files.

1. **`for index, row in df.iterrows():`**
   - This line starts a loop over each row in the DataFrame `df`.
   - `df.iterrows()` is a pandas function that allows you to loop through the DataFrame row by row.
   - `index` represents the row number (starting from 0), and `row` contains the data for that specific row in the form of a pandas Series.

2. **`file_name = f"{path}/text_file_{index + 1}.txt"`**
   - This line generates a unique filename for each row.
   - The `f"{path}/text_file_{index + 1}.txt"` uses an f-string to create a file name based on the `index` (plus 1 to make it start at 1 instead of 0).
   - `path` is a variable that contains the directory where the file will be saved (it should be defined earlier in the code).
   - For example, for the first row (index 0), this would create a filename like `"path/to/directory/text_file_1.txt"`.

3. **`with open(file_name, 'w') as file:`**
   - This opens a file with the name `file_name` in write mode (`'w'`), allowing the program to write data into it.
   - The `with` statement ensures the file is properly closed after writing, even if an error occurs.

4. **`text_content = str(row['CombinedText']) if pd.notnull(row['CombinedText']) else ''`**
   - This checks the value in the `'CombinedText'` column for the current row.
   - **`pd.notnull(row['CombinedText'])`** checks if the value is **not** `NaN` (i.e., it’s not missing).
   - If the value is **not** `NaN`, it converts the value to a string using `str(row['CombinedText'])`.
   - If the value is `NaN`, it sets `text_content` to an empty string (`''`).
   - This ensures that no matter what value is in the `CombinedText` column, you will have valid text to write to the file (either the text itself or an empty string).

5. **`file.write(text_content)`**
   - This writes the `text_content` (the string version of the `CombinedText` value) to the file.
   - If the `CombinedText` was `NaN`, it writes an empty string; otherwise, it writes the actual content from that column.




In [ ]:
for index, row in df.iterrows():
    file_name = f"{path}/factiva/text_file_{index + 1}.txt"  # Create a unique filename for each row
    with open(file_name, 'w') as file:
        text_content = str(row['CombinedText']) if pd.notnull(row['CombinedText']) else ''  # Convert to string and handle NaN
        file.write(text_content)  # Write the text content to the file